In [1]:
import anndata
import torch
import stPlus

import squidpy as sq
import numpy as np
import scanpy as sc
import pandas as pd

from sklearn.model_selection import KFold
from transpa.eval_util import calc_corr
from transpa.util import expTransImp, leiden_cluster, compute_autocorr
from benchmark import SpaGE_impute, Tangram_impute
import warnings

warnings.filterwarnings('ignore')

seed = 10
device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

Global seed set to 0


In [2]:
import os
merfish_path = '../data/merfish.h5ad'
if os.path.exists(merfish_path):
    spa_adata = sc.read(merfish_path)
else:
    merfish = pd.read_csv('/data/users/cqiao/data/stPlus/data/SpaGE Datasets/Spatial/MERFISH/Moffitt_and_Bambah-Mukku_et_al_merfish_all_cells.csv')
    merfish_1 = merfish.loc[merfish['Animal_ID'] == 1, :]
    merfish_1 = merfish_1.loc[merfish_1['Cell_class'] != 'Ambiguous',:]
    merfish_meta = merfish_1.iloc[:,0:9]
    merfish_data = merfish_1.iloc[:,9:171]
    merfish_data = merfish_data.drop(columns = ['Blank_1','Blank_2','Blank_3','Blank_4','Blank_5','Fos'])

    spa_adata = anndata.AnnData(merfish_data.values)
    spa_adata.var_names = merfish_data.columns.values
    spa_adata.var_names_make_unique()
    spa_adata.obs = merfish_meta
    spa_adata.obs['X'] = merfish_1.Centroid_X.values
    spa_adata.obs['Y'] = merfish_1.Centroid_Y.values
    sc.pp.normalize_total(spa_adata)
    sc.pp.log1p(spa_adata)
    spa_adata.write('../data/merfish.h5ad')


Moffit_path = '../data/moffit_adata.h5ad'

if os.path.exists(Moffit_path):
    Moffit_adata = sc.read(Moffit_path)
else:
    Moffit_adata = sc.read_mtx("/data/users/cqiao/data/stPlus/data/SpaGE Datasets/scRNAseq/Moffit_RNA/GSE113576/matrix.mtx").T
    genes = pd.read_csv('/data/users/cqiao/data/stPlus/data/SpaGE Datasets/scRNAseq/Moffit_RNA/GSE113576/genes.tsv',sep='\t',header=None).loc[:, 1].values
    barcodes = pd.read_csv('/data/users/cqiao/data/stPlus/data/SpaGE Datasets/scRNAseq/Moffit_RNA/GSE113576/barcodes.tsv',sep='\t',header=None).loc[:, 0].values

    Moffit_adata.var_names = genes
    Moffit_adata.obs_names = barcodes
    Moffit_adata.var_names_make_unique()
    classes, ct_list = leiden_cluster(Moffit_adata)
    cls_key = 'leiden'
    Moffit_adata.obs[cls_key] = classes
    sc.pp.filter_genes(Moffit_adata, min_cells=10)
    sc.pp.normalize_total(Moffit_adata)
    sc.pp.log1p(Moffit_adata)
    Moffit_adata.write('../data/moffit_adata.h5ad')
    
spa_adata.var_names_make_unique()
Moffit_adata.var_names_make_unique()
ct_list = np.unique(Moffit_adata.obs.leiden)
classes = Moffit_adata.obs.leiden.values
cls_key = 'leiden'

In [3]:
raw_spatial_df  = pd.DataFrame(spa_adata.X, columns=spa_adata.var_names)
# raw_scrna_df    = pd.DataFrame.sparse.from_spmatrix(Moffit_adata.X, columns=Moffit_adata.var_names)
raw_scrna_df    = pd.DataFrame(Moffit_adata.X.toarray(), columns=Moffit_adata.var_names)
raw_shared_gene = np.intersect1d(raw_spatial_df.columns, raw_scrna_df.columns)

raw_spatial_df.to_csv('output/merfish_raw.csv')
raw_spatial_df.shape, raw_scrna_df.shape, raw_shared_gene.shape

((64373, 155), (31299, 18646), (153,))

In [4]:
spa_adata.obsm['spatial'] = np.hstack([spa_adata.obs.X.values.reshape(-1,1), spa_adata.obs.Y.values.reshape(-1,1)])
np.save('output/merfish_locations.npy', spa_adata.obsm['spatial'])
sq.gr.spatial_neighbors(spa_adata)

In [5]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score, homogeneity_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

n_clusters = len(spa_adata.obs.Cell_class.unique())
print(f"{n_clusters} clusters")
# real_clss = AgglomerativeClustering(n_clusters=n_clusters, 
#                                     connectivity=spa_adata.obsp['spatial_connectivities'],
#                                     ).fit_predict(StandardScaler().fit_transform(spa_adata.X))
real_clss = spa_adata.obs.Cell_class.values

15 clusters


In [6]:
def spa_agglomerative(true_imp_X):
    pred_clss = AgglomerativeClustering(n_clusters=n_clusters, 
                                        connectivity=spa_adata.obsp['spatial_connectivities'],
                                       ).fit_predict(StandardScaler().fit_transform(true_imp_X))
    ars = adjusted_rand_score(real_clss, pred_clss)
    amis = adjusted_mutual_info_score(real_clss, pred_clss)
    homo = homogeneity_score(real_clss, pred_clss)
    nmi = normalized_mutual_info_score(real_clss, pred_clss)
    scores = {"ARS":ars, "AMIS":amis, 'HOMO': homo, 'NMI':nmi}
    print(f"Cluster on Raw Expression vs  on Predicted Expression\n - ARS: {ars:.6f}, AMIS: {amis:.6f}, HOMO: {homo:.6f}, NMI: {nmi:.6f}")
    scores = {"ARS":ars, "AMIS":amis, 'HOMO': homo, 'NMI':nmi}
    return scores

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(raw_shared_gene)

df_transImpSpa = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImpCls = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImpClsSpa = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImp = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_stplus_res = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_spaGE_res = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_tangram_res = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)


spa_cluster_metrics = []
method_names = []
for idx, (train_ind, test_ind) in enumerate(kf.split(raw_shared_gene)):    
    print(f"\n===== Fold {idx+1} =====\nNumber of train genes: {len(train_ind)}, Number of test genes: {len(test_ind)}")
    train_gene = raw_shared_gene[train_ind]
    test_gene  = raw_shared_gene[test_ind]
    
    test_spatial_df = raw_spatial_df[test_gene]
    spatial_df = raw_spatial_df[train_gene]
    scrna_df   = raw_scrna_df

    df_transImpSpa[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            signature_mode='cell',
            mapping_mode='lowrank',
            mapping_lowdim=128,
            wt_spa=1,
            spa_adj=spa_adata.obsp['spatial_connectivities'].tocoo(),
            seed=seed,
            device=device)
    
    spa_cluster_metrics.append(spa_agglomerative(np.hstack([df_transImpSpa[test_gene].values, raw_spatial_df[train_gene].values])))
    method_names.append("transImpSpa")

    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpSpa, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImpSpa)')

    df_transImpCls[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            ct_list=ct_list,
            classes=classes,
            signature_mode='cluster',
            mapping_mode='full',
            seed=seed,
            device=device)

    spa_cluster_metrics.append(spa_agglomerative(np.hstack([df_transImpCls[test_gene].values, raw_spatial_df[train_gene].values])))
    method_names.append('transImpCls')
    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpCls, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImpCls)')

    df_transImp[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            signature_mode='cell',
            mapping_mode='lowrank',
            mapping_lowdim=128,
            seed=seed,
            device=device)

    spa_cluster_metrics.append(spa_agglomerative(np.hstack([df_transImp[test_gene].values, raw_spatial_df[train_gene].values])))
    method_names.append("transImp")
    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImp, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImp)')

    df_transImpClsSpa[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            ct_list=ct_list,
            classes=classes,
            spa_adj=spa_adata.obsp['spatial_connectivities'].tocoo(),
            signature_mode='cluster',
            mapping_mode='full',
            wt_spa=1,
            seed=seed,
            device=device)
    
    spa_cluster_metrics.append(spa_agglomerative(np.hstack([df_transImpClsSpa[test_gene].values, raw_spatial_df[train_gene].values])))
    method_names.append("transImpClsSpa")
    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpClsSpa, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImpClsSpa)')

    
    df_stplus_res[test_gene] = stPlus.stPlus(spatial_df, scrna_df, test_gene, "tmp_mm", verbose=False, random_seed=seed, device=device)
    spa_cluster_metrics.append(spa_agglomerative(np.hstack([df_stplus_res[test_gene].values, raw_spatial_df[train_gene].values])))
    method_names.append("stPlus")
    corr_res_stplus = calc_corr(raw_spatial_df, df_stplus_res, test_gene)
    print(f'\t\t\t{np.median(corr_res_stplus)} (stPlus)')

    df_spaGE_res[test_gene]  = SpaGE_impute(scrna_df, spatial_df, train_gene, test_gene)
    spa_cluster_metrics.append(spa_agglomerative(np.hstack([df_spaGE_res[test_gene].values, raw_spatial_df[train_gene].values])))
    method_names.append("spaGE")
    corr_res_spaGE = calc_corr(raw_spatial_df, df_spaGE_res, test_gene)
    print(f'\t\t\t{np.median(corr_res_spaGE)} (spaGE)')

    df_tangram_res[test_gene] = Tangram_impute(Moffit_adata, spa_adata, train_gene, test_gene, device, cls_key)
    spa_cluster_metrics.append(spa_agglomerative(np.hstack([df_tangram_res[test_gene].values, raw_spatial_df[train_gene].values])))
    method_names.append("tangram")
    corr_res_tangram = calc_corr(raw_spatial_df, df_tangram_res, test_gene)
    print(f'\t\t\t{np.median(corr_res_tangram)} (Tangram)')

corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpSpa, raw_shared_gene)
corr_transImp_res = calc_corr(raw_spatial_df, df_transImp, raw_shared_gene)
corr_transImpCls_res = calc_corr(raw_spatial_df, df_transImpCls, raw_shared_gene)
corr_transImpClsSpa_res = calc_corr(raw_spatial_df, df_transImpClsSpa, raw_shared_gene)
corr_res_stplus = calc_corr(raw_spatial_df, df_stplus_res, raw_shared_gene)
corr_res_spaGE = calc_corr(raw_spatial_df, df_spaGE_res, raw_shared_gene)
corr_res_tangram = calc_corr(raw_spatial_df, df_tangram_res, raw_shared_gene)   

print(np.median(corr_transImpSpa_res), "(TransImpSpa)", 
      np.median(corr_transImp_res), "(TransImp)", 
      np.median(corr_transImpCls_res), "(TransImpCls)", 
      np.median(corr_transImpClsSpa_res), "(TransImpClsSpa)", 
      np.median(corr_res_stplus), "(stPlus)", 
      np.median(corr_res_spaGE), "(spaGE)",
      np.median(corr_res_tangram), "(Tangram)"
      )


===== Fold 1 =====
Number of train genes: 122, Number of test genes: 31


[TransImp] Epoch: 1000/1000, loss: 0.562223, (IMP) 0.560559, (SPA) 1 x 0.001664: 100%|██████████| 1000/1000 [00:18<00:00, 55.00it/s]


Cluster on Raw Expression vs  on Predicted Expression
 - ARS: 0.048491, AMIS: 0.088764, HOMO: 0.095563, NMI: 0.089448
fold 0, median correlation: 0.33793506902181053 (TransImpSpa)


[TransImp] Epoch: 1000/1000, loss: 0.843272, (IMP) 0.843272, (SPA) 1.0 x 0.000000: 100%|██████████| 1000/1000 [00:07<00:00, 133.62it/s]


Cluster on Raw Expression vs  on Predicted Expression
 - ARS: 0.051081, AMIS: 0.081695, HOMO: 0.091842, NMI: 0.082358
fold 0, median correlation: 0.3571415145959039 (TransImpCls)


...

Cluster on Raw Expression vs  on Predicted Expression
 - ARS: 0.041138, AMIS: 0.078593, HOMO: 0.089842, NMI: 0.079246
fold 4, median correlation: 0.27403045692845285 (TransImpClsSpa)
Cluster on Raw Expression vs  on Predicted Expression
 - ARS: 0.072289, AMIS: 0.094932, HOMO: 0.093392, NMI: 0.095677
			0.2356610237098042 (stPlus)
Cluster on Raw Expression vs  on Predicted Expression
 - ARS: 0.048119, AMIS: 0.080144, HOMO: 0.089574, NMI: 0.080812
			0.22733023498071797 (spaGE)


INFO:root:123 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:123 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 123 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.437, KL reg: 0.101
Score: 0.661, KL reg: 0.002
Score: 0.662, KL reg: 0.002
Score: 0.663, KL reg: 0.002
Score: 0.663, KL reg: 0.002
Score: 0.663, KL reg: 0.002
Score: 0.663, KL reg: 0.002
Score: 0.663, KL reg: 0.002
Score: 0.663, KL reg: 0.002
Score: 0.663, KL reg: 0.002


INFO:root:Saving results..


Cluster on Raw Expression vs  on Predicted Expression
 - ARS: 0.044704, AMIS: 0.077923, HOMO: 0.088828, NMI: 0.078581
			0.2830945919730508 (Tangram)
0.27322788572156775 (TransImpSpa) 0.2783631214199657 (TransImp) 0.28280039215836916 (TransImpCls) 0.2801339827156346 (TransImpClsSpa) 0.26358982611465287 (stPlus) 0.25811461426728405 (spaGE) 0.2893536322874469 (Tangram)


In [8]:
df_transImp.to_csv('output/merfish_moffit_transImpute.csv')
df_transImpSpa.to_csv('output/merfish_moffit_transImpSpa.csv')
df_transImpCls.to_csv('output/merfish_moffit_transImpCls.csv')
df_transImpClsSpa.to_csv('output/merfish_moffit_transImpClsSpa.csv')
df_spaGE_res.to_csv('output/merfish_moffit_spaGE.csv')
df_stplus_res.to_csv('output/merfish_moffit_stPlus.csv')
df_tangram_res.to_csv('output/merfish_moffit_Tangram.csv')

df = pd.DataFrame(spa_cluster_metrics)
df['method'] = method_names
df.to_csv("output/clustering_5fold/merfish_moffit.csv")


In [2]:
df = pd.read_csv("output/clustering_5fold/merfish_moffit.csv", index_col=0)
df.groupby('method').mean()

,ARS,AMIS,HOMO,NMI
method,,,,
spaGE,0.039564,0.074565,0.082379,0.075245
stPlus,0.045572,0.082171,0.089317,0.082855
tangram,0.041625,0.075826,0.085092,0.076494
transImp,0.045357,0.078954,0.087845,0.079624
transImpCls,0.049668,0.080347,0.089345,0.081018
transImpClsSpa,0.046616,0.078892,0.087313,0.079566
transImpSpa,0.045435,0.083965,0.092097,0.084641


In [10]:
dict_df = {"TransImp":df_transImp, 
           "TransImpSpa":df_transImpSpa, 
           "TransImpCls":df_transImpCls,
           "TransImpClsSpa":df_transImpClsSpa,
           "spaGE": df_spaGE_res, "stPlus": df_stplus_res, 
           "Tangram":df_tangram_res
           }
sq.gr.spatial_autocorr(
    spa_adata,
    n_jobs=10,
)
sq.gr.spatial_autocorr(
    spa_adata,
    n_jobs=10,
    mode='geary',
)

dict_adata = {name: compute_autocorr(spa_adata[:, raw_shared_gene].copy(), df) for name, df in dict_df.items()}


In [11]:
from sklearn.metrics import mean_squared_error
moranIs = {name:mean_squared_error(spa_adata.uns['moranI'].loc[raw_shared_gene].I, imp_adata.uns['moranI'].loc[raw_shared_gene].I) for name, imp_adata in dict_adata.items()}
gearyCs = {name:mean_squared_error(spa_adata.uns['gearyC'].loc[raw_shared_gene].C, imp_adata.uns['gearyC'].loc[raw_shared_gene].C) for name, imp_adata in dict_adata.items()}

print("Mean Squared Error\nMoran's I:\n")
print("\n".join([f"\tTrue vs {method}: {score:.6f}" for method, score in moranIs.items()]))
print("Geary's C:\n")
print("\n".join([f"\tTrue vs {method}: {score:.6f}" for method, score in gearyCs.items()]))


Mean Squared Error
Moran's I:

	True vs TransImp: 0.008554
	True vs TransImpSpa: 0.005353
	True vs TransImpCls: 0.007834
	True vs TransImpClsSpa: 0.004871
	True vs spaGE: 0.003916
	True vs stPlus: 0.005143
	True vs Tangram: 0.008349
Geary's C:

	True vs TransImp: 0.008628
	True vs TransImpSpa: 0.005370
	True vs TransImpCls: 0.007751
	True vs TransImpClsSpa: 0.004840
	True vs spaGE: 0.003970
	True vs stPlus: 0.005206
	True vs Tangram: 0.008244
